In [ ]:
# This cell tells python to use the local version of CVToolkit instead of the installed one.
# Use for testing purposes until branch feature/BCV-970-oor-metrics is merged.
import sys
import os

module_path = os.path.abspath(os.path.join("../../CVToolkit"))
if module_path not in sys.path:
    sys.path.insert(0, module_path)

In [ ]:
import sys
import pandas as pd

sys.path.append("../..")

from objectherkenning_openbare_ruimte.performance_evaluation_pipeline.metrics.per_pixel_stats import (
    EvaluatePixelWise,
)
from objectherkenning_openbare_ruimte.performance_evaluation_pipeline.metrics.metrics_utils import (
    ObjectClass,
    BoxSize,
)

In [ ]:
# Input: ground truth and predictions
gt_path = "../../datasets/oor/eval_metrics_test/labels/ground_truth"
pred_path = "../../datasets/oor/eval_metrics_test/labels/predictions"

# Size of the images (width, height)
img_shape = (1024, 1024)

In [ ]:
# Compute performance over classes and bounding box sizes

evaluator = EvaluatePixelWise(gt_path, pred_path, img_shape)


# Compute the Total Blurred Area results for the person & licence plate classes

tba_results = evaluator.collect_results_per_class_and_size(
    classes=[ObjectClass.person, ObjectClass.license_plate],
    box_sizes=BoxSize
)
evaluator.store_tba_results(results=tba_results)

In [ ]:
# Compute per pixel results 

per_pixel_results = evaluator.collect_results_per_class_and_size(
    classes=ObjectClass,
    box_sizes=[BoxSize.all]
)

pd.DataFrame(data=per_pixel_results).transpose()

In [ ]:
from typing import Any, Dict, List, Tuple, Union
from torch import tensor, empty

def cxcywh_norm_to_xywh(cxcywh: Tuple[float, float, float, float], image_shape: Tuple[int, int]) -> Tuple[float, float, float, float]:
    (img_width, img_height) = image_shape
    x_center, y_center, width, height = cxcywh
    x_center, y_center, width, height = (
        x_center * img_width,
        y_center * img_height,
        width * img_width,
        height * img_height,
    )
    x_min = (x_center - width / 2)
    y_min = (y_center - height / 2)
    return x_min, y_min, width, height

def yolo_to_cl_xywh(yolo_annotation: str, image_shape: Tuple[int, int]) -> Tuple[int, Tuple[float, float, float, float]]:
    cls, cx, cy, w, h = map(
        float, yolo_annotation.split()
    )
    return int(cls), cxcywh_norm_to_xywh((cx, cy, w, h), image_shape)

def yolo_pred_to_cl_xywh_conf(yolo_annotation: str, image_shape: Tuple[int, int]) -> Tuple[int, Tuple[float, float, float, float], float]:
    # cls, cx, cy, w, h, conf = map(
    #     float, yolo_annotation.split()
    # )
    cls, cx, cy, w, h = map(
        float, yolo_annotation.split()
    )
    conf = 0.536
    return int(cls), cxcywh_norm_to_xywh((cx, cy, w, h), image_shape), conf

def yolo_annotation_file_to_torchmetrics_dict(yolo_annotation_file: Union[str, os.PathLike], image_shape: Tuple[int, int], is_pred: bool = False) -> Dict[str, Any]:
    with open(yolo_annotation_file, mode='r') as f:
        boxes = []
        labels = []
        scores = []
        for line in f.readlines():
            if is_pred:
                cls, box, conf = yolo_pred_to_cl_xywh_conf(line, image_shape)
                boxes.append(box)
                labels.append(cls)
                scores.append(conf)
            else:
                cls, box = yolo_to_cl_xywh(line, image_shape)
                boxes.append(box)
                labels.append(cls)
        torch_dict = {
            'boxes': tensor(boxes),
            'labels': tensor(labels),
        }
        if is_pred:
            torch_dict['scores'] = tensor(scores)
        return torch_dict

In [ ]:

import pathlib

gt_annotations = pathlib.Path(gt_path).glob("*.txt")

gt_data = []
pred_data = []

for gt_file in gt_annotations:
    if gt_file.stem.startswith("._"):
        continue

    gt_dict = yolo_annotation_file_to_torchmetrics_dict(gt_file, image_shape=img_shape)
    gt_dict['name'] = gt_file.stem
    gt_data.append(gt_dict)

    pred_file = os.path.join(pred_path, gt_file.name)
    if os.path.isfile(pred_file):
        pred_dict = yolo_annotation_file_to_torchmetrics_dict(pred_file, image_shape=img_shape, is_pred=True)
    else:
        pred_dict = {
            'boxes': empty((0, 4)),
            'labels': empty((0)),
            'scores': empty((0)),
        }
    pred_dict['name'] = gt_file.stem
    pred_data.append(pred_dict)


In [ ]:
from pprint import pprint

ind = 2

pprint(gt_data[ind])
pprint(pred_data[ind])

In [ ]:
from torchmetrics.detection import MeanAveragePrecision

metric = MeanAveragePrecision(box_format="xywh", iou_type="bbox", class_metrics=True)
metric.update(pred_data, gt_data)

pprint(metric.compute())